# Proyecto de algoritmo de análisis de sentimiento difuso a partir de tweets.

### Autor: Victor Ulises Lovera Duarte

-----

## Paso 1:
Para el primer paso tenemos que leer y hacer un preprocesado de los datos. Para la lectura usamos la librería de **pandas** que, no solo tiene una función para extraer los datos directamente del csv, si no que también es una herramienta que nos facilitara el procesamiento de los datos y su posterior exposición.
Luego pasamos al preprocesado de los datos para estos usamos la librería de **re** que nos ayuda a quitar los componentes de texto que no queremos, además de remplazar alguna de las palabras acortadas comúnmente usadas, por ultimo pasamos todo a minúsculas, esto hacemos por cada oración en los datos.

In [8]:
import pandas as pd

def Leer_Archivo(Archivo):
    data = pd.read_csv(Archivo)
    data.insert(0, 'Id', range(1, 1 + len(data)))
    return data

In [9]:
import re

def Pre_proceso_Texto(data):
    def process(text):
        
        text = re.sub(r'http[s]?://\S+', '', text)
        text = re.sub(r' www\S+', '', text)
        text = re.sub(r'@\S+', '', text)
        text = re.sub(r'[^\w\s]|[\d]', ' ', text)
        text = re.sub(r'\s\s+', ' ', text)
        text = re.sub(r"#", "", text)
        
        text = re.sub(r"n\'t", " not", text)
        text = re.sub(r"\'re", " are", text)
        text = re.sub(r"\'s", " is", text)
        text = re.sub(r"\'d", " would", text)
        text = re.sub(r"\'ll", " will", text)
        text = re.sub(r"\'t", " not", text)
        text = re.sub(r"\'ve", " have", text)
        text = re.sub(r"\'m", " am", text)
        
        text = text.strip().lower().encode('ascii', 'ignore').decode()
        
        return text

    for i, row in data.iterrows():
        row['sentence'] = process(row['sentence'])
        
    return data

----

## Paso 2:
Ahora debemos de realizar el análisis del Lexicón de Sentimientos, para esto usamos una librería llamada **NLTK** que nos provee con las funciones necesarias para lograrlo. Llegados a este punto también comenzamos calculando el tiempo que nos lleva analizar el Lexicón de Sentimientos de cada oración y los incluimos como una columna más en los datos.

In [10]:
import time

from nltk.sentiment import SentimentIntensityAnalyzer
def Lexicon_Sentiminetos(data):
    sia = SentimentIntensityAnalyzer()

    res = {}
    tiempos = []
    for i, row in data.iterrows():
        start_time = time.time()
        
        text = row['sentence']
        myid = row['Id']
        res[myid] = sia.polarity_scores(text)
        
        tiempos.append(time.time() - start_time)

    Datos_Sent = pd.DataFrame(res).T
    Datos_Sent = Datos_Sent.reset_index().rename(columns={'index': 'Id'})
    Datos_Sent['Tiempo Computo'] = tiempos
    
    Datos_Sent = Datos_Sent.merge(data, how='left').drop(['compound', 'neu'], axis=1)
    
    return Datos_Sent


----

## Paso 3:
En este punto empezamos delimitando el Universo de nuestras funcionen difusas, así como también las funciones de membresía de las varias funciones que vamos a usar estas son:
1. **Sent_Postivo**: Universo de los resultados positivos del algoritmo anterior
   1. **sp_lo**
   2. **sp_med**
   3. **sp_hi**
2. **Sent_Negativo**: Universo de los resultados negativos del algoritmo anterior
   1. **sn_lo**
   2. **sn_med**
   3. **sn_hi**
3. **Out_U**: Universo de la salida de las reglas que serán aplicados
   1. **out_neg**
   2. **out_neu**
   3. **out_pos**

In [11]:
import numpy as np
import skfuzzy as fuzz

Sent_Postivo = np.arange(0, 1, 0.01)
Sent_Negativo = np.arange(0, 1, 0.01)
Out_U = np.arange(0, 10, 1)

sp_lo = fuzz.trimf(Sent_Postivo, [0, 0, 0.5])
sp_med = fuzz.trimf(Sent_Postivo, [0, 0.5, 1])
sp_hi = fuzz.trimf(Sent_Postivo, [0.5, 1, 1])

sn_lo = fuzz.trimf(Sent_Negativo, [0, 0, 0.5])
sn_med = fuzz.trimf(Sent_Negativo, [0, 0.5, 1])
sn_hi = fuzz.trimf(Sent_Negativo, [0.5, 1, 1]) 

out_neg = fuzz.trimf(Out_U, [0, 0, 5])
out_neu = fuzz.trimf(Out_U, [0, 5, 10])
out_pos = fuzz.trimf(Out_U, [5, 5, 10])

Una vez optemos los datos del análisis del Lexicón de Sentimientos, más específicos los valores puntajes *positivos* y *negativos* los usamos juntos a las reglas propuestas por [Vashishtha S., & Susan, S. (2019). **Fuzzy rule based unsupervised sentiment analysis from social media posts**](https://www.researchgate.net/profile/Srishti-Vashishtha-2/publication/334622166_Fuzzy_Rule_based_Unsupervised_Sentiment_Analysis_from_Social_Media_Posts/links/5ece42174585152945149e5b/Fuzzy-Rule-based-Unsupervised-Sentiment-Analysis-from-Social-Media-Posts.pdf), y por ultimo utilizamos el método de deFuzzification del Centro de Gravedad para final mente hallar la métrica que nos determina si la oración es positiva o negativa.

In [12]:
def Fuzzy_DeFuzzy(scoreposi, scoreneg):
    
    scoreposi = max(0, min(1, scoreposi))
    scoreneg = max(0, min(1, scoreneg))
    
    pos_n_lo = fuzz.interp_membership(Sent_Postivo, sp_lo, scoreposi)
    pos_n_med = fuzz.interp_membership(Sent_Postivo, sp_med, scoreposi)
    pos_n_hi = fuzz.interp_membership(Sent_Postivo, sp_hi, scoreposi)

    neg_n_lo = fuzz.interp_membership(Sent_Negativo, sn_lo, scoreneg)
    neg_n_med = fuzz.interp_membership(Sent_Negativo, sn_med, scoreneg)
    neg_n_hi = fuzz.interp_membership(Sent_Negativo, sn_hi, scoreneg)

    regla1 = np.fmin(pos_n_lo, neg_n_lo)
    regla2 = np.fmin(pos_n_med, neg_n_lo) 
    regla3 = np.fmin(pos_n_hi, neg_n_lo) 
    regla4 = np.fmin(pos_n_lo, neg_n_med) 
    regla5 = np.fmin(pos_n_med, neg_n_med) 
    regla6 = np.fmin(pos_n_hi, neg_n_med) 
    regla7 = np.fmin(pos_n_lo, neg_n_hi) 
    regla8 = np.fmin(pos_n_med, neg_n_hi) 
    regla9 = np.fmin(pos_n_hi, neg_n_hi)

    temp1 = np.fmax(regla4, regla7)
    regla_neg = np.fmax(temp1, regla8)
    out_act_low = np.fmin(regla_neg, out_neg)

    temp2 = np.fmax(regla1, regla5)
    regla_neu = np.fmax(temp2, regla9)
    out_act_neu = np.fmin(regla_neu, out_neu)

    temp3 = np.fmax(regla2, regla3)
    regla_pos = np.fmax(temp3, regla6)
    out_act_pos = np.fmin(regla_pos, out_pos)

    aggregated = np.fmax(out_act_low,
                        np.fmax(out_act_neu, out_act_pos))

    return fuzz.defuzz(Out_U, aggregated, 'centroid')

----

Ahora ya solo queda seguir todos los pasos, con esta función retornamos el DataFrame ya con los resultados y los costes temporales resultante para cada oración.

In [13]:
def Computar_Datos(Archivo):
    
    
    Datos = Leer_Archivo(Archivo)
    
    Datos = Pre_proceso_Texto(Datos)
    
    Datos = Lexicon_Sentiminetos(Datos)
    
    Resultados = Datos.copy()

    tiempos = []
    for i, row in Datos.iterrows():
        start_time = time.time()
        
        Resultados.loc[i, 'Results'] = Fuzzy_DeFuzzy(row['pos'], row['neg'])
        
        tiempos.append(time.time() - start_time)

    Resultados['Tiempo Computo'] = Resultados['Tiempo Computo'] + tiempos
    
    Resultados.drop('Id', axis=1)
    Resultados = Resultados[['sentence', 'sentiment', 'pos', 'neg', 'Results', 'Tiempo Computo']]
    Resultados.columns = ['Oración', 'Etiqueta original', 'Puntaje Positivo', 'Puntaje Negativo', 'Resultado de inferencia', 'Tiempo Computo']

    return Resultados


In [14]:
Archivo = './Data/test_data.csv'
    
Resultados = Computar_Datos(Archivo)

Resultados.head()

,Oración,Etiqueta original,Puntaje Positivo,Puntaje Negativo,Resultado de inferencia,Tiempo Computo
0,i loooooooovvvvvveee my kindle not that the dx...,1,0.382,0.000,5.371795,0.002509
1,reading my kindle love it lee childs is good read,1,0.470,0.000,5.963203,0.000000
2,ok first assesment of the kindle it fucking rocks,1,0.216,0.000,4.890017,0.000999
3,you ll love your kindle i ve had mine for a fe...,1,0.204,0.135,4.685033,0.001006
4,fair enough but i have the kindle and i think ...,1,0.456,0.000,5.849136,0.000000


In [15]:
print("El Promedio temporal de Ejecucion es:", Resultados['Tiempo Computo'].mean())

El Promedio temporal de Ejecucion es: 0.00042291604044709697


In [16]:
def Total_Pos_Neg(data):
    
    pos_t = 0
    neg_t = 0
    
    for i, row in data.iterrows():
        if row['Resultado de inferencia'] < 5:
            neg_t += 1
        else:
            pos_t += 1
        
    return pos_t, neg_t

In [17]:
Total_Positivos, Total_Negativos = Total_Pos_Neg(Resultados)
print("\nLa cantidad de Oraciones Positivas es: ", Total_Positivos)
print("\nLa cantidad de Oraciones Negativas es: ", Total_Negativos)


La cantidad de Oraciones Positivas es:  98

La cantidad de Oraciones Negativas es:  261
